In [ ]:
%run vis/xAPISG-noDataToFillVisualization.ipynb # notebook to create the visualisation with a message NoDataToFill
%run vis/helpersFunctions/barCharts.ipynb
%run vis/helpersFunctions/OrderGraphValuesAndSelectDataByXaxis.ipynb

***vis_completables_scores* function**:
* Input: players_info dictionary 
* Displays bar chart showing the score achieved by players in the different completables

In [2]:
outputCompletableScore=widgets.Output()
def vis_completables_scores():
    out=interactive(getAllScorePerCompletable,
                       #constructing a dropdown to select how many data we want to see in visualisations for x axis
                      xaxis=widgets.Dropdown(options=[('All datas',0),('5 per visualisation',5),('25 per visualisation',25),('50 per visualisation',50),('10 first',-1),('10 last',-2)],
                                description='x axe',
                                disabled=False),
                       #constructing a dropdown to order for alphabetic keys or for values
                        order=widgets.Dropdown(options=[('alphabetic',0),('value from highest to lowest',1),('value from lowest to highest',-1)],
                                description='Order',
                                disabled=False))
    md_text=widgets.HTML("<h4>Bar chart with scores of players in each completable</h4>")
    with outputCompletableScore:
        print("Updating visualisation with the current selection of players and completables...")
    return widgets.VBox([md_text,out, outputCompletableScore])

def getAllScorePerCompletable(xaxis, order):
    #get all players selected
    playersSelected=checkboxesPlayersSelected.get_selected_options()
    completableSelected=getSublistSelectedFromUnionListSelected(completable_score_list,
                                                                checkboxesCompletableSelected.get_selected_options())
    players_scores = {}
    for player in playersSelected:
        if players_info[player]["completables_scores"] != []:
            for c in completableSelected:
                if not c in players_scores.keys():
                    players_scores[c]={}
                if c in players_info[player]["completables_scores"].keys():
                    try:
                        players_scores[c][player] = int(players_info[player]["completables_scores"][c])
                    except:
                        players_scores[c][player] = 0
                        for value in players_info[player]["completables_scores"][c].values():
                            players_scores[c][player]+=value
                else:
                    players_scores[c][player] = 0
    ordered_scores = orderDictByDictValues(players_scores,order)
    pos, bar_width = getPosDictMultiBarChart(ordered_scores,playersSelected)
    nbgraphshown, players_scores, N = organizeDictPerxAxisSelected(completableSelected,xaxis,
                                                                   players_scores)
    pos = organizeDictPosPerDataxaxis(xaxis,pos)
    for n in range(nbgraphshown):
        n_groups = len(players_scores[n])
        players_scores[n] = reorganiseDictBySubKeys(players_scores[n])
        if len(completableSelected)>0 and len(playersSelected)>0:
                constructMultiBarWithVerticalLines(players_scores,n,
                                                   playersSelected, completableSelected,
                                                   pos, bar_width, N)
        else:
            noDataToFillVis(100)
        plt.xlabel("Completable")
        plt.ylabel("Score")
        plt.xticks(rotation=90)
        plt.title("Scores of players per completable")
        plt.show()
    with outputCompletableScore:
        clear_output(wait=False)